# Incubyte - Task

Read data and insert it into tabes as per given specification.

Using Python, Pandas and SQLite

## Setup

In [1]:
# list package versions
!pip list | grep 'pandas\|sqllite3'

pandas                        1.1.5         
pandas-datareader             0.9.0         
pandas-gbq                    0.13.3        
pandas-profiling              1.4.1         
sklearn-pandas                1.8.0         


In [2]:
# data directory
!ls 'drive/MyDrive/Incubyte'

'Data Engineer Assessment.pdf'	 sample.txt


In [3]:
# import packages
import pandas as pd
import sqlite3

In [4]:
path_to_file = 'drive/MyDrive/Incubyte/sample.txt'

## Specification

In [5]:
# column names
columns = ['Customer_Name', 'Customer_Id', 'Open_Date', 'Last_Consulted_Date', 
           'Vaccination_Id', 'Dr_Name', 'State', 'Country', 'Post_Code',
           'DOB', 'Is_Active']

In [6]:
# sample data file
!cat 'drive/MyDrive/Incubyte/sample.txt'

|H|Customer_Name|Customer_Id|Open_Date|Last_Consulted_Date|Vaccination_Id|Dr_Name|State|Country|DOB|Is_Active
|D|Alex|123457|20101012|20121013|MVD|Paul|SA|USA|06031987|A
|D|John|123458|20101012|20121013|MVD|Paul|TN|IND|06031987|A
|D|Mathew|123459|20101012|20121013|MVD|Paul|WAS|PHIL|06031987|A
|D|Matt|12345|20101012|20121013|MVD|Paul|BOS|NYC|06031987|A
|D|Jacob|1256|20101012|20121013|MVD|Paul|VIC|AU|06031987|A


In [7]:
# query to create table, if it doeasnt already exists

create_table_query = """
                     CREATE TABLE IF NOT EXISTS {table_name} (
                         Customer_Name VARCHAR(255) NOT NULL PRIMARY KEY,
                         Customer_Id VARCHAR(18) NOT NULL,
                         Open_Date DATE(8) NOT NULL,
                         Last_Consulted_Date DATE(8),
                         Vaccination_Id CHAR(5),
                         Dr_Name CHAR(255),
                         State CHAR(5),
                         Country CHAR(5),
                         Post_Code INT(5),
                         DOB DATE(8),
                         Is_Active CHAR(1)
                     )
                     """

In [8]:
# Create a test table and check specification

DB_NAME = 'hospital_chain.db'
TABLE_NAME = 'Table_XXXXX'

# init db connection
with sqlite3.connect(DB_NAME) as db:
    cur = db.cursor()
    
    cur.execute(create_table_query.format(table_name=TABLE_NAME))

    table_info = cur.execute("pragma table_info({0})".format(TABLE_NAME))
    for column in table_info.fetchall():
        print(column)

(0, 'Customer_Name', 'VARCHAR(255)', 1, None, 1)
(1, 'Customer_Id', 'VARCHAR(18)', 1, None, 0)
(2, 'Open_Date', 'DATE(8)', 1, None, 0)
(3, 'Last_Consulted_Date', 'DATE(8)', 0, None, 0)
(4, 'Vaccination_Id', 'CHAR(5)', 0, None, 0)
(5, 'Dr_Name', 'CHAR(255)', 0, None, 0)
(6, 'State', 'CHAR(5)', 0, None, 0)
(7, 'Country', 'CHAR(5)', 0, None, 0)
(8, 'Post_Code', 'INT(5)', 0, None, 0)
(9, 'DOB', 'DATE(8)', 0, None, 0)
(10, 'Is_Active', 'CHAR(1)', 0, None, 0)


All the columns are as per specification. Now we can move onto processing data.

## Read data

In [9]:
# just python
with open(path_to_file) as f:
    for line in f.readlines():
        print(line)

|H|Customer_Name|Customer_Id|Open_Date|Last_Consulted_Date|Vaccination_Id|Dr_Name|State|Country|DOB|Is_Active

|D|Alex|123457|20101012|20121013|MVD|Paul|SA|USA|06031987|A

|D|John|123458|20101012|20121013|MVD|Paul|TN|IND|06031987|A

|D|Mathew|123459|20101012|20121013|MVD|Paul|WAS|PHIL|06031987|A

|D|Matt|12345|20101012|20121013|MVD|Paul|BOS|NYC|06031987|A

|D|Jacob|1256|20101012|20121013|MVD|Paul|VIC|AU|06031987|A



In [10]:
# pandas
df = pd.read_csv(path_to_file, sep='|', header=0, usecols=range(2, 12),
                 names=['Name', 'Cust_I', 'Open_Dt', 'Consul_Dt', 'VAC_ID', 'DR_Name', 'State', 'Country', 'DOB', 'FLAG'])

df

,Name,Cust_I,Open_Dt,Consul_Dt,VAC_ID,DR_Name,State,Country,DOB,FLAG
0,Alex,123457,20101012,20121013,MVD,Paul,SA,USA,6031987,A
1,John,123458,20101012,20121013,MVD,Paul,TN,IND,6031987,A
2,Mathew,123459,20101012,20121013,MVD,Paul,WAS,PHIL,6031987,A
3,Matt,12345,20101012,20121013,MVD,Paul,BOS,NYC,6031987,A
4,Jacob,1256,20101012,20121013,MVD,Paul,VIC,AU,6031987,A


## Load data

In [11]:
# run this to remove db, ignore if it doesn't exist
!rm 'hospital_chain.db'

In [12]:
# query to insert data to table, ignore if any constraint isn't satisified
insert_query = """
               INSERT OR IGNORE INTO {table_name} VALUES (?, ?, ?, ?, ?, ?, ?, ?, NULL, ?, ?)
               """

In [13]:
# split data by country, create tables and insert data

with sqlite3.connect(DB_NAME) as db:
    cur = db.cursor()
    
    for country in df.Country.unique():
        TABLE_NAME = 'Table_' + country
        print('Inserting into: ', TABLE_NAME)
        
        cur.execute(create_table_query.format(table_name=TABLE_NAME))

        rows = df[df.Country==country].values
        cur.executemany(insert_query.format(table_name=TABLE_NAME), rows)

Inserting into:  Table_USA
Inserting into:  Table_IND
Inserting into:  Table_PHIL
Inserting into:  Table_NYC
Inserting into:  Table_AU


In [14]:
# inspect data in db

with sqlite3.connect(DB_NAME) as db:
    cur = db.cursor()
    
    for country in df.Country.unique():
        TABLE_NAME = 'Table_' + country

        cur.execute("SELECT COUNT(1) FROM {}".format(TABLE_NAME))
        print('\n{name} : {records} records'.format(name=TABLE_NAME, records=cur.fetchone()[0]))

        print(cur.execute("SELECT * FROM {}".format(TABLE_NAME)).fetchall())


Table_USA : 1 records
[('Alex', '123457', 20101012, 20121013, 'MVD', 'Paul', 'SA', 'USA', None, 6031987, 'A')]

Table_IND : 1 records
[('John', '123458', 20101012, 20121013, 'MVD', 'Paul', 'TN', 'IND', None, 6031987, 'A')]

Table_PHIL : 1 records
[('Mathew', '123459', 20101012, 20121013, 'MVD', 'Paul', 'WAS', 'PHIL', None, 6031987, 'A')]

Table_NYC : 1 records
[('Matt', '12345', 20101012, 20121013, 'MVD', 'Paul', 'BOS', 'NYC', None, 6031987, 'A')]

Table_AU : 1 records
[('Jacob', '1256', 20101012, 20121013, 'MVD', 'Paul', 'VIC', 'AU', None, 6031987, 'A')]
